In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import re

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)  



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
There is no [win32] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Mark A\.wdm\drivers\chromedriver\win32\97.0.4692.71]


In [3]:
url = 'https://www.brewersassociation.org/directories/breweries/?location=GA'
browser.visit(url)

In [4]:
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [5]:
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [6]:
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [7]:
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [8]:
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [9]:
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

# Convert the browser html to a soup object
html = browser.html
breweries = soup(html, 'html.parser')

In [10]:
membership =breweries.find_all('div', class_='logos')


In [11]:
len(membership)

174

In [12]:
#Scrape member types: 
member_type = []
craft =[] 

member_str = 'https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2019/05/BA_member-1.png'
craft_seal = 'https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2018/12/BA_Independent_Seal-2.svg'
ba = 'https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2019/05/ba_associate.png'


for i in membership:
    mark= i.find_all('img')
    if len(mark)==2:
        temp = []
        for j in mark: 
            temp.append(j['src'])
        if temp[0] == member_str:
            member_type.append('Beer Association Member')
            craft.append('Independent Craft Brewer')
        elif temp[0] == ba: 
            craft.append('Independent Craft Brewer')
            member_type.append('Beer Association Associate Member')
    elif len(mark)==1:
        temp = []
        for j in mark: 
            temp.append(j['src'])
        if temp[0] == member_str:
            member_type.append('Beer Association Member')
        elif temp[0] == craft_seal: 
            craft.append('Independent Craft Brewer')
            member_type.append('No Membership')
        else: 
            member_type.append('Beer Association Associate Member')
            craft.append('No Seal')
    else: 
        member_type.append('No Membership')
        craft.append('No Seal')
    

In [13]:
company_listing= breweries.find_all('div', class_='company-listing')
company_listing

[<div class="company-listing"><div class="company-content" itemscope="" itemtype="https://schema.org/LocalBusiness"><h3 class="with-mini-hr" itemprop="name">52 Brewing</h3><div itemprop="address" itemscope="" itemtype="https://schema.org/PostalAddress"><p class="alt mb-0"><span itemprop="addressLocality">Dahlonega</span>, <span itemprop="addressRegion">GA</span> 30533</p></div><p class="alt mb-0">Type: Planning</p><a href="http://www.google.com/maps/place/350 Highway 52 West+Dahlonega+GA+USA" target="_blank">View Map</a></div><div class="logos"><img alt="" src="https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2019/05/ba_associate.png"/></div></div>,
 <div class="company-listing shaded"><div class="company-content" itemscope="" itemtype="https://schema.org/LocalBusiness"><h3 class="with-mini-hr" itemprop="name">6S Brewing Company</h3><div itemprop="address" itemscope="" itemtype="https://schema.org/PostalAddress"><p class="alt mb-0" itemprop="streetAddress">3111 Main S

In [14]:
#brewery type 
b_type = ["Taproom", "Brewpub", "Micro", "Regional", "Contract","Proprietor"]
#initializing arrays for populating 
company_name = []
company_type =[]
company_phone=[]
street_address =[]
city =[]
state =[]
zipcode =[]

for company in company_listing: 
    #scrape company name
    company_name.append(company.find('h3', class_='with-mini-hr').text)
    #print(company.find('h3', class_='with-mini-hr').text)
    #scrape company type
    typeA = company.find_all('p', text = re.compile("Type:"))
    state.append('GA')
    if len(typeA)==1:
        typeA = company.find('p', text = re.compile("Type:")).text
        temp_type = typeA.split(':')
        temp_type = temp_type[1]
        temp_type = temp_type.strip()
        company_type.append(temp_type)
    else: 
        company_type.append(company.find('a',text=b_type).text)
    
    #scrape address
    if len(company.find('div', itemprop ="address"))==2: #for situations where there are street address and other info 
        ad = company.find('div', itemprop ="address")
        ad = ad.find_all('p')
        street_address.append(ad[0].text)
        a = ad[1].text 
        a = a.split('GA')
        city.append(a[0])
        zipcode.append(a[1].strip())
    else:
        ad = company.find('div', itemprop ="address")
        ad = ad.find_all('p')
        ad = ad[0].text
        ad = ad.split('GA')
        city.append(ad[0].split(',')[0])
        zipcode.append(ad[1].strip())
        street_address.append('N/A')
    
    #scrape phone number
    if company.find('span', itemprop="telephone") != None: 
        company_phone.append(company.find('span', itemprop="telephone").text)
    else: 
        company_phone.append('no phone number')

In [15]:
#transform to dictionary to convert to pandas object 
beer_dict = {
    "company_name": company_name, 
    "company_type": company_type,
    "company_phone": company_phone, 
    "street_address": street_address, 
    "city": city, 
    "state": state, 
    "zipcode": zipcode,
    "member_type": member_type,
    "independent_craft": craft
}

In [16]:
beerDB = pd.DataFrame(beer_dict)

In [17]:
beerDB = beerDB.set_index("company_name")

In [18]:
beerDB

,company_type,company_phone,street_address,city,state,zipcode,member_type,independent_craft
company_name,,,,,,,,
52 Brewing,Planning,no phone number,N/A,Dahlonega,GA,30533,Beer Association Associate Member,No Seal
6S Brewing Company,Taproom,(678) 597-8335,3111 Main Street,"Duluth,",GA,30096,Beer Association Member,Independent Craft Brewer
Abide Brewing Company,Micro,(678) 972-6747,110 Werz Industrial Blvd Ste 130,"Newnan,",GA,30263-5802,Beer Association Member,Independent Craft Brewer
Akademia Brewing Company,Brewpub,(706) 461-3208,150 Crane Dr,"Athens,",GA,30606-0600,No Membership,Independent Craft Brewer
Amici At Trillith,Brewpub,(678) 892-5858,355 Trilith Parkway STE 110,"Fayetteville,",GA,30214,No Membership,Independent Craft Brewer
...,...,...,...,...,...,...,...,...
Whistle Top Brew Company,Taproom,(706) 776-9556,248 N Main St,"Cornelia,",GA,30531,No Membership,Independent Craft Brewer
Wild Heaven - West End Brewery & Gardens,Brewpub,(404) 997-8589,1010 White St SW,"Atlanta,",GA,30310,Beer Association Member,Independent Craft Brewer
Wild Heaven Beer,Micro,(404) 997-8589,135B Maple St,"Decatur,",GA,30030-3953,Beer Association Member,Independent Craft Brewer


In [19]:
beerDB.to_csv("Data/beer_association_scrape.csv")